In [1]:
import openai
import time
import json

def extract_message_content(message):
    # Extract the message content
    message_content = message.content[0].text
    annotations = message_content.annotations

    # Iterate over the annotations and add footnotes
    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote
        print(annotation.text)
        message_content.value = message_content.value.replace(annotation.text, ' ')
    
    return  message_content.value

client = openai.OpenAI(api_key="这里替换成你的OpenAI API Key")

# 上传客服知识文件
file_knowledge_call_center = client.files.create(
    file=open("niubiclean-book.txt", "rb"),
    # 文件用途，目前取值为：fine-tune、assistants
    purpose="assistants"
)

assistant_call_center = client.beta.assistants.create(
    name="牛逼净化器智能客服",
    # 功能描述，最长 512 字符
    description="24小时为您服务",
    # 功能prompt，最长 32768 字符
    instructions="你是牛逼净化器公司的智能客服，请引用文件中的内容回答问题，表达要通俗易懂、尽量简短；若问题超出文件内容，请委婉拒绝。",
    model="gpt-3.5-turbo-1106",
    # assistants开启的工具，共有三种类型：code_interpreter、retrieval、function
    tools=[
        {
            # gpt提供的检索功能，若上传了知识文件，则可以查询，类似于智能客服/知识库
            "type": "retrieval",
        }
    ],
    # 知识文件，通过File接口上传的
    file_ids=[file_knowledge_call_center.id]
)

thread_zhangsan = client.beta.threads.create(
    metadata={
        "姓名": "用户甲",
        "年龄": 36,
        "性别": "男"
    }
)

# 我的三个问题
questions = ["净化器有什么功能？","怎么更换滤网？","你们的联系电话是什么？","今天天气怎么样？"]

# 遍历询问我的问题
for question in questions: 
    print("用户甲：",question)

    message = client.beta.threads.messages.create(
        thread_id=thread_zhangsan.id,  
        role="user",          
        content=question,
    )

    # 创建咨询队列
    run = client.beta.threads.runs.create(
        thread_id=thread_zhangsan.id,
        assistant_id=assistant_call_center.id,
    )

    # 等待询问结果
    while run.status == "queued" or run.status == "in_progress":
        # 免费账号的使用频率有限制，这里多等会  
        #time.sleep(20)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_zhangsan.id,
            run_id=run.id,
        )
    
    #print(run.status)
    
    if run.status=="failed":
        print(run.last_error.message)
    else:
        # 获取询问结果
        messages = client.beta.threads.messages.list(
            thread_id=thread_zhangsan.id, order="asc", after=message.id
        )
        #display(json.loads(messages.model_dump_json()))
        print("牛逼智能客服：",extract_message_content(messages.data[0]),'\n')

    # 免费账号的使用频率有限制，这里多等会   
    #time.sleep(40)

用户甲： 净化器有什么功能？
【7†source】
牛逼智能客服： 净化器的功能包括：

1. 多重净化能力：可有效过滤PM2.5颗粒物和吸附甲醛、苯、TVOC等有害气体。
2. 多档位风速调节：提供静音、低速、中速、高速和自动调节档位。
3. 智能自动模式：内置空气质量传感器，能实时监测空气质量并根据结果自动调整工作模式和风速。
4. 夜间模式：在夜间可以调至超静音状态，LED显示灯会变暗或关闭，确保安静舒适的睡眠环境。
5. WiFi智能连接：支持Wi-Fi无线连接，可以通过专用APP远程控制和监控设备，实现开关机、档位调整、定时预约、滤网更换提醒等功能 。 

用户甲： 怎么更换滤网？
【12†source】
牛逼智能客服： 要更换滤网，可以按照以下步骤操作：

1. 当APP提示或机身指示灯亮起时，请更换相应的滤网。
2. 关闭电源，按照产品手册中的步骤打开滤网舱门，取出旧滤网，装入新的滤网，并确保其正确安装到位 。 

用户甲： 你们的联系电话是什么？
牛逼智能客服： 你可以联系我们的客户服务热线400-88888，或者查阅官方网站www.niubiclean.com获取更多支持信息【1†source】。 

用户甲： 今天天气怎么样？
牛逼智能客服： 抱歉，我无法回答关于天气的问题。如果您需要了解当地天气情况，建议您联系当地气象台或使用天气预报软件进行查询。 

